In [15]:
import advent

data = advent.get_lines(7)

In [16]:
def hand_to_dict(hand: str):
    result: dict[str, int] = {}
    for h in hand:
        if h not in result:
            result[h] = 1
        else:
            result[h] += 1
    return result

    
def hand_type(hand: str):
    hand_d = hand_to_dict(hand)
    hand_values = sorted(hand_d.values())[::-1]
    if hand_values[0] == 5: return 7
    elif hand_values[0] == 4: return 6
    elif hand_values[0] == 3 and hand_values[1] == 2: return 5
    elif hand_values[0] == 3: return 4
    elif hand_values[0] == 2 and hand_values[1] == 2: return 3
    elif hand_values[0] == 2: return 2
    else: return 1

def compare_card(card1: str, card2: str, part1:bool=True):
    if part1:
        cards = '23456789TJQKA'
    else:
        cards = 'J23456789TQKA'
    return cards.index(card1) - cards.index(card2)

compare_card_2 = lambda card1, card2: compare_card(card1, card2, False)

def compare(hand1: str, hand2: str):
    # positive: hand1 is better
    # negative: hand2 is better
    ht1, ht2 = hand_type(hand1), hand_type(hand2)
    if ht1 > ht2: return 1
    if ht1 < ht2: return -1
    for ix in range(len(hand2)):
        cmp = compare_card(hand1[ix], hand2[ix])
        if cmp != 0: return cmp
    raise ValueError("Two hands are the same!")

assert compare('33222', '22334') > 0
assert compare('34422', '24444') < 0
assert compare('54433', '2AAKK') > 0
assert compare('AA2QQ', 'AA5KK') < 0


In [17]:
def compare_lines(line1: str, line2: str):
    hand1, hand2 = line1.split(" ")[0], line2.split(" ")[0]
    return compare(hand1, hand2)

from functools import cmp_to_key
hands_sorted = sorted(data, key=cmp_to_key(compare_lines))

# add 1 because the first (worst) hand gets 1 point, not 0
sum([(ix+1) * int(line.split(" ")[1]) for ix, line in enumerate(hands_sorted)])

249390788

In [18]:
# hopefully this works: joker feels like it can 'simply' be assigned as follows:
# take the 'hand dictionary', get the most common card (in case of ties, pick the highest)
# it doesnt seem like you would want to do something more complicated (e.g. have jokers take different values)

def apply_joker(hand: str):
    if 'J' not in hand: return hand
    hand_dict = hand_to_dict(hand)
    del hand_dict['J']
    if hand_dict == {}: return 'AAAAA' # special case
    most_common = sorted(hand_dict.values())[-1]
    most_common_cards = [h for h in hand_dict if hand_dict[h] == most_common]
    to_replace = sorted(most_common_cards, key=cmp_to_key(compare_card_2))[-1]
    return hand.replace('J', to_replace)

apply_joker('JAATTJ')


'AAATTA'

In [21]:
# compare_part2 is 95% the same as compare. but I didn't know how to cleanly refactor compare
# without making that funciton quite ugly, so I decided to just copy it

# only real difference is that the input is now a tuple

def compare_part2(hand1, hand2):
    hand1, orig_hand1 = hand1
    hand2, orig_hand2 = hand2

    ht1, ht2 = hand_type(hand1), hand_type(hand2)
    if ht1 > ht2: return 1
    if ht1 < ht2: return -1
    for ix in range(len(orig_hand2)):
        cmp = compare_card(orig_hand1[ix], orig_hand2[ix], part1=False)
        if cmp != 0: return cmp
    raise ValueError("Two hands are the same!")

def compare_lines_part2(line1, line2):
    orig_hand1, orig_hand2 = line1.split(" ")[0], line2.split(" ")[0]
    hand1 = apply_joker(orig_hand1)
    hand2 = apply_joker(orig_hand2)
    return compare_part2((hand1, orig_hand1), (hand2, orig_hand2))

hands_sorted = sorted(data, key=cmp_to_key(compare_lines_part2))

# add 1 because the first (worst) hand gets 1 point, not 0
sum([(ix+1) * int(line.split(" ")[1]) for ix, line in enumerate(hands_sorted)])

248750248